# Expression analysis

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import dash_bio
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn3
import scipy.signal as signal
import venn

In this notebook, we'll have a look at some general patterns of expression of the assembled transcripts.

In [ ]:
# Load the data
expression = pd.read_csv('../../data/kallisto/tpm.csv', engine='pyarrow', index_col=0)
transcripts_per_L = pd.read_csv('../../data/kallisto/transcripts_per_L.csv', engine='pyarrow', index_col=0)

# Load the metadata
meta = pd.read_csv('../../samples.csv')

In [ ]:
expression.head()

Let's start by exploring the expression distribution.

In [ ]:
# Calculate row sums
sum = expression.sum(axis=1).sort_values(ascending=False)
sum_transcripts_per_L = transcripts_per_L.sum(axis=1).sort_values(ascending=False)
not_expressed = len(sum[sum == 0])
lowly_expressed = len(sum[sum < 1])
total = len(sum)
print(f'In total {len(expression)} transcripts have been assembled')
print(f'{(not_expressed / total) * 100} percent of assembled transcripts are not detected in TPM')
print(f'{(lowly_expressed / total) * 100} percent of assembled transcripts are not expressed above 1 TPM in all samples')

### Annotation of transcripts

Let's plot the percentage of assembled transcripts for which we found some annotation information.

In [ ]:
# Load the functional annotation data
functional_annotation = pd.read_csv('../../data/annotation/functional_eggnog/functional_annotation.emapper.annotations', engine='pyarrow', sep='\t')
# Fix transcript names in the first column so that they equal the transcript identifiers in the count files
functional_annotation.iloc[:, 0] = functional_annotation.iloc[:, 0].str.split(".", expand=True).drop(columns=1)

# Load taxonomic information
taxonomy = pd.read_table('../../data/annotation/taxonomy_phyloDB_extended/phylodb_1.076.taxonomy_extended.txt')
annotation = pd.read_table('../../data/annotation/taxonomy_phyloDB_extended/phylodb_1.076.annotations_extended.txt', engine='pyarrow', header=None)
annotation.columns = ['target_id', 'code', 'strain_name', 'function']
# load the annotation alignment data
taxonomic_annotation_90 = pd.read_table('../../data/annotation/taxonomy_phyloDB_extended/phylodb_extended.firsthit.90plus_alnscore.m8', engine='pyarrow', header=None)
taxonomic_annotation_60 = pd.read_table('../../data/annotation/taxonomy_phyloDB_extended/phylodb_extended.firsthit.60plus_alnscore.m8', engine='pyarrow', header=None)

taxonomic_annotation_90.columns = ['query_id', 'target_id', 'p_ident', 'alnlen', 'mismatch', 'gapopen', 'qstart', 'qend', 'tstart', 'tend', 'evalue', 'bits']
# Fix transcript names in the first column so that they equal the transcript identifiers in the count files
taxonomic_annotation_90.iloc[:, 0] = taxonomic_annotation_90.iloc[:, 0].str.split(".", expand=True).drop(columns=1)
# Add the taxonomy information to the taxonomic_annotation_90
# taxonomic_annotation_90 = taxonomic_annotation_90.merge(annotation, left_on='target_id', right_on='target_id', how='left')
taxonomic_annotation_90 = taxonomic_annotation_90.merge(annotation, left_on='target_id', right_on='target_id')
taxonomic_annotation_90 = taxonomic_annotation_90.merge(taxonomy, left_on='strain_name', right_on = 'strain_name')
taxonomic_annotation_90 = taxonomic_annotation_90.drop(columns=['code', 'peptide_count', 'mismatch', 'gapopen', 'qstart', 'qend', 'tstart', 'tend', 'evalue', 'bits'])
# Expand taxonomy path in relevant columns
taxonomic_annotation_90[['kingdom', 'superphylum', 'phylum', 'class', 'order', 'family', 'genus', 'species']] = taxonomic_annotation_90['taxonomy'].str.split(';', expand = True)


taxonomic_annotation_60.columns = ['query_id', 'target_id', 'p_ident', 'alnlen', 'mismatch', 'gapopen', 'qstart', 'qend', 'tstart', 'tend', 'evalue', 'bits']
# Fix transcript names in the first column so that they equal the transcript identifiers in the count files
taxonomic_annotation_60.iloc[:, 0] = taxonomic_annotation_60.iloc[:, 0].str.split(".", expand=True).drop(columns=1)
# Add the taxonomy information to the taxonomic_annotation_60
# taxonomic_annotation_60 = taxonomic_annotation_60.merge(annotation, left_on='target_id', right_on='target_id', how='left')
taxonomic_annotation_60 = taxonomic_annotation_60.merge(annotation, left_on='target_id', right_on='target_id')
taxonomic_annotation_60 = taxonomic_annotation_60.merge(taxonomy, left_on='strain_name', right_on = 'strain_name')
taxonomic_annotation_60 = taxonomic_annotation_60.drop(columns=['code', 'peptide_count', 'mismatch', 'gapopen', 'qstart', 'qend', 'tstart', 'tend', 'evalue', 'bits'])
# Expand taxonomy path in relevant columns
taxonomic_annotation_60[['kingdom', 'superphylum', 'phylum', 'class', 'order', 'family', 'genus', 'species']] = taxonomic_annotation_60['taxonomy'].str.split(';', expand = True)

# Load the EukProt annotation data
taxonomic_annotation_90_eukprot = pd.read_table('../../data/annotation/taxonomy_eukprot/eukprot_DB.firsthit.90plus_alnscore.m8', engine='pyarrow', header=None)
taxonomic_annotation_60_eukprot = pd.read_table('../../data/annotation/taxonomy_eukprot/eukprot_DB.firsthit.60plus_alnscore.m8', engine='pyarrow', header=None)

# Fix transcript names in the first column so that they equal the transcript identifiers in the count files
taxonomic_annotation_90_eukprot.iloc[:, 0] = taxonomic_annotation_90_eukprot.iloc[:, 0].str.split(".", expand=True).drop(columns=1)
taxonomic_annotation_60_eukprot.iloc[:, 0] = taxonomic_annotation_60_eukprot.iloc[:, 0].str.split(".", expand=True).drop(columns=1)

## In the second column, split of the EukProt ID off
eukprot_ID = taxonomic_annotation_90_eukprot.iloc[:, 1].str.split("_", expand=True)[0]
taxonomic_annotation_90_eukprot.iloc[:, 1] = eukprot_ID
taxonomic_annotation_90_eukprot.columns = ['query_id', 'target_id', 'p_ident', 'alnlen', 'mismatch', 'gapopen', 'qstart', 'qend', 'tstart', 'tend', 'evalue', 'bits']

eukprot_ID = taxonomic_annotation_60_eukprot.iloc[:, 1].str.split("_", expand=True)[0]
taxonomic_annotation_60_eukprot.iloc[:, 1] = eukprot_ID
taxonomic_annotation_60_eukprot.columns = ['query_id', 'target_id', 'p_ident', 'alnlen', 'mismatch', 'gapopen', 'qstart', 'qend', 'tstart', 'tend', 'evalue', 'bits']

## Add taxonomic information
eukprot_taxonomy = pd.read_table('../../data/annotation/taxonomy_eukprot/EukProt_included_data_sets.v03.2021_11_22.txt')
print(f'The eukprot taxonomy file contains {len(eukprot_taxonomy)} rows')

# Drop the columns that are not needed
eukprot_taxonomy.drop(columns=['Previous_Names', 'Replaces_EukProt_ID', 'Data_Source_URL', 'Data_Source_Name', 'Paper_DOI', 'Actions_Prior_to_Use',
       'Data_Source_Type', 'Notes', 'Columns_Modified_Since_Previous_Version', 'Merged_Strains',
       'Alternative_Strain_Names', '18S_Sequence_GenBank_ID', '18S_Sequence',
       '18S_Sequence_Source', '18S_Sequence_Other_Strain_GenBank_ID',
       '18S_Sequence_Other_Strain_Name', '18S_and_Taxonomy_Notes'], inplace=True)

# Swap the _ to a space in the Name_to_Use column
eukprot_taxonomy['Name_to_Use'] = eukprot_taxonomy['Name_to_Use'].str.replace('_', ' ')

# Merge the annotation and taxonomy files
taxonomic_annotation_90_eukprot = taxonomic_annotation_90_eukprot.merge(eukprot_taxonomy, left_on='target_id', right_on='EukProt_ID', how='left')
taxonomic_annotation_60_eukprot = taxonomic_annotation_60_eukprot.merge(eukprot_taxonomy, left_on='target_id', right_on='EukProt_ID', how='left')

# Drop the columns that are not needed
taxonomic_annotation_90_eukprot.drop(columns=['target_id', 'gapopen', 'qstart', 'qend', 'tstart', 'tend', 'evalue', 'bits'], inplace=True)
taxonomic_annotation_60_eukprot.drop(columns=['target_id', 'gapopen', 'qstart', 'qend', 'tstart', 'tend', 'evalue', 'bits'], inplace=True)

In [ ]:
taxonomic_annotation_60.head()

In [ ]:
taxonomic_annotation_60_eukprot.head()

In [ ]:
# Check whether the transcripts in the expression data have a certain annotation

## Get the total number of predicted proteins or transcripts
total_transcripts = len(expression)
total_proteins = 3705883

## Create a dataframe to store the results
annotation_found = pd.DataFrame(columns=['annotated', 'unannotated', 'total_proteins'])

## Define the annotations of interest
annotation_of_interest = ['KEGG_ko', 'PFAMs', 'GOs', 'EC', 'KEGG_Pathway', 'KEGG_Module', 'max_annot_lvl', 'COG_category']

## Populate the dataframe
for annotation in annotation_of_interest:
    annotated = len(functional_annotation[functional_annotation[annotation] != '-'][annotation].notna())
    unannotated = total_proteins - annotated
    annotation_found.loc[annotation] = [annotated, unannotated, total_proteins]

# Do the same thing for the taxonomic annotation
annotation_of_interest = ['taxonomy']

for annotation in annotation_of_interest:
    # 90% identity
    annotated = len(taxonomic_annotation_90[taxonomic_annotation_90[annotation] != '-'][annotation].notna())
    unannotated = total_proteins - annotated
    annotation_found.loc[annotation + '_90'] = [annotated, unannotated, total_proteins]
    # 60% identity
    annotated = len(taxonomic_annotation_60[taxonomic_annotation_60[annotation] != '-'][annotation].notna())
    unannotated = total_proteins - annotated
    annotation_found.loc[annotation + '_60'] = [annotated, unannotated, total_proteins]

# EukProt 90
annotated = len(taxonomic_annotation_90_eukprot[taxonomic_annotation_90_eukprot['Name_to_Use'] != '-']['Name_to_Use'].notna())
unannotated = total_proteins - annotated
annotation_found.loc['eukprot_90'] = [annotated, unannotated, total_proteins]

# EukProt 60
annotated = len(taxonomic_annotation_60_eukprot[taxonomic_annotation_60_eukprot['Name_to_Use'] != '-']['Name_to_Use'].notna())
unannotated = total_proteins - annotated
annotation_found.loc['eukprot_60'] = [annotated, unannotated, total_proteins]
    
# Transform the results into percentage, drop the total_proteins column and rename the index column
annotation_found['annotated'] = annotation_found['annotated'] / annotation_found['total_proteins'] * 100
annotation_found['unannotated'] = annotation_found['unannotated'] / annotation_found['total_proteins'] * 100
annotation_found = annotation_found.drop(columns=['total_proteins'])
# Reshape the dataframe into a long format
annotation_found = annotation_found.reset_index()
annotation_found = annotation_found.melt(id_vars='index', var_name='annotation', value_name='percentage')
annotation_found.rename(columns={'index': 'category'}, inplace=True)

In [ ]:
fig = px.bar(annotation_found,
            x='category', y='percentage', color='annotation',
            color_discrete_map={'unannotated': 'gray', 'annotated': 'blue'},
            barmode='stack')

fig.update_layout(
    font=dict(
        family="Times New Roman, serif",  # Set the font family to Times New Roman
        size=12,  # Set the font size
        color="#7f7f7f"  # Set the font color
    ),
    autosize=False,
    # Convert the width and height from centimeters to pixels
    width=int(8 / 2.54 * 96),  # 1 inch = 2.54 cm, 96 pixels per inch
    height=int(7 / 2.54 * 96),
    margin=dict( # Set the margins
        l=0,  # Left margin
        r=25,  # Right margin
        b=25,  # Bottom margin
        t=25  # Top margin
    ),
    showlegend=False,
    yaxis_title='% annotated',
    xaxis_tickangle=-90
)

fig.show()

# Save the figure
fig.write_image('../../figures/assembly/annotation.svg')

In [ ]:
# Print the results
print(annotation_found)

Let's plot the overlap of proteins predicted from the metatranscriptome that have both taxonomic and functional annotations, only one or none.

In [ ]:
# Get the set of transcript IDs for each dataframe
set1 = set(expression.index)
# Read in predicted protein names
set2 = pd.read_table('../../data/assembly/protein/SPAdes_mmseqsDB.lookup', header=None)[1]
# Cut off the strings that got appended to the transcript names
set2 = set([x.split('.')[0] for x in set2])

# Create the figure and axes objects
fig, ax = plt.subplots(figsize=(4,4))

# Plot the Venn diagram
venn2([set1, set2], ('all', 'predicted proteins'),  ax=ax)

# Save the figure as an SVG file
plt.savefig('../../figures/assembly/transcript_protein_venn.svg', format='svg', dpi=1200)

plt.show()

In [ ]:
taxonomic_annotation_60_eukprot.head()

In [ ]:
# Get the set of transcript IDs for each dataframe
set1 = set(expression.index)
set2 = set([x.split('.')[0] for x in pd.read_table('../../data/assembly/protein/SPAdes_mmseqsDB.lookup', header=None)[1]])
set3 = set(taxonomic_annotation_60['query_id'])
set4 = set(functional_annotation['#query'])
set5 = set(taxonomic_annotation_60_eukprot['query_id'])
set6 = set(taxonomic_annotation_90['query_id'])
set7 = set(taxonomic_annotation_90_eukprot['query_id'])

labels = venn.get_labels([set2, set3, set4, set5], fill=['number', 'logic'])

# Create the figure and axes objects
fig, ax = venn.venn2(labels, names=['predicted proteins', 'taxonomic_phyloDB_60', 'functional', 'taxonomic_eukprot_60'])

# Save the figure as an SVG file
plt.savefig('../../figures/assembly/protein_annotation_venn.svg', format='svg', dpi=1200)

# Plot the Venn diagram
fig.show()

In [ ]:
labels = venn.get_labels([set3, set4, set5], fill=['number', 'logic'])

# Create the figure and axes objects
fig, ax = venn.venn2(labels, names=['taxonomic_phyloDB_60', 'functional', 'taxonomic_eukprot_60'])

# Save the figure as an SVG file
plt.savefig('../../figures/assembly/annotation_venn.svg', format='svg', dpi=1200)

# Plot the Venn diagram
fig.show()

In [ ]:
# Same setup, different figure
# Create the figure and axes objects
fig, ax = plt.subplots(figsize=(4,4))

# Plot the Venn diagram
venn3([set3, set4, set5], ('taxonomic_phyloDB_60', 'functional', 'taxonomic_eukprot_60'),  ax=ax)

# Save the figure as an SVG file
plt.savefig('../../figures/assembly/annotation_venn_2.svg', format='svg', dpi=1200)

plt.show()

In [ ]:
labels = venn.get_labels([set4, set6, set7], fill=['number', 'logic'])

# Create the figure and axes objects
fig, ax = venn.venn2(labels, names=['functional', 'taxonomic_phyloDB_90', 'taxonomic_eukprot_90'])

# Save the figure as an SVG file
plt.savefig('../../figures/assembly/annotation_venn_90.svg', format='svg', dpi=1200)

# Plot the Venn diagram
fig.show()

## Transcript length distribution

The sequnece length file was generated using the get_sequence_lengths.rs script. It is a single-column containing the nucleotide length of every sequence. Let's visualize the distribution of sequence lengths.

In [ ]:
# Read the sequence length data (a random sample of 10% of the sequences)
sequence_length = pd.read_table('../../data/analysis/metatranscriptome_sequence_lengths.txt', header=None)

# Read the sequence length data (a random sample of 10% of the sequences)
# sequence_length = pd.read_table('../../data/analysis/metatranscriptome_sequence_lengths.txt', header=None,
#                                 skiprows=lambda i: i>0 and np.random.rand() > 0.5)
sequence_length.columns = ['length']

# Plot the distribution of sequence lengths using plotly
fig = px.histogram(sequence_length, x='length', log_y=True, nbins=1000, opacity=0.5)

fig.update_layout(
    font=dict(
        family="Times New Roman, serif",  # Set the font family to Times New Roman
        size=8
    ),
    autosize=False,
    # Convert the width and height from centimeters to pixels
    width=int(9 / 2.54 * 96),  # 1 inch = 2.54 cm, 96 pixels per inch
    height=int(7 / 2.54 * 96),
    margin=dict( # Set the margins
        l=0,  # Left margin
        r=25,  # Right margin
        b=25,  # Bottom margin
        t=25  # Top margin
    ),
    yaxis_title='Log number of transcripts',
)

fig.show()

# Save the figure
fig.write_image('../../figures/assembly/sequence_length.svg')

In [ ]:
# Log transform the data
sequence_length['length_log'] = np.log10(sequence_length['length'])

# Plot the distribution of sequence lengths using plotly
fig = px.histogram(sequence_length, x='length_log', log_y=False, nbins=1000, opacity=0.5)

fig.update_layout(
    font=dict(
        family="Times New Roman, serif",  # Set the font family to Times New Roman
        size=8
    ),
    autosize=False,
    # Convert the width and height from centimeters to pixels
    width=int(9 / 2.54 * 96),  # 1 inch = 2.54 cm, 96 pixels per inch
    height=int(7 / 2.54 * 96),
    margin=dict( # Set the margins
        l=0,  # Left margin
        r=25,  # Right margin
        b=25,  # Bottom margin
        t=25  # Top margin
    ),
    yaxis_title='Number of transcripts',
    xaxis_title='Log transcript length (bp)'
)

# Add vertie(x=sequence_length['length_log'].median(), line_width=1, line_dash='dash', line_color='black')
# Add value of tcal lines for the median sequence length
fig.add_vlinhe most counted sequence length
fig.add_annotation(x=sequence_length['length_log'].median(), y=1, text='Median transcript length: 342 bp', 
                    showarrow=False, textangle=-90, yshift=110, xshift=10, font=dict(size=10))

fig.show()

# Save the figure
fig.write_image('../../figures/assembly/sequence_length_log.svg')

In [ ]:
# Print out summary statistics
sequence_length = pd.read_table('../../data/analysis/metatranscriptome_sequence_lengths.txt', header=None)
sequence_length.columns = ['length']
print('Number of transcripts in the metatranscriptome: ' + str(len(sequence_length)))
print('Mean transcript length: ' + str(sequence_length['length'].mean()))
print('Median transcript length: ' + str(sequence_length['length'].median()))
print('Maximum transcript length: ' + str(sequence_length['length'].max()))
print('Minimum transcript length: ' + str(sequence_length['length'].min()))

## Expression vs Variance

In [ ]:
# Extract the squared coefficient of variance from each row
coeff_var = (np.std(expression, axis=1) / np.mean(expression, axis=1)) ** 2
# Extract the mean expression from each row
mean_expression = np.mean(expression, axis=1)
# Combine both values in a dataframe
df = pd.DataFrame()
df['CV2'] = coeff_var
df['CV'] = np.sqrt(coeff_var)
df['mean_expression'] = mean_expression
df['log_mean_expression'] = np.log10(mean_expression + 1)

# Subset the dataframe to only include transcripts with a mean expression of at least 1 TPM
df = df[df['mean_expression'] >= 1]

# Subsample the dataframe to only include 50% of the data for computational efficiency
df = df.sample(frac=0.5)

fig = px.scatter(df, x='log_mean_expression', 
                y='CV', opacity=0.2)

fig.update_layout(
    font=dict(
        family="Times New Roman, serif",  # Set the font family to Times New Roman
        size=12,  # Set the font size
        color="#7f7f7f"  # Set the font color
    ),
    autosize=False,
    # Convert the width and height from centimeters to pixels
    width=int(9 / 2.54 * 96),  # 1 inch = 2.54 cm, 96 pixels per inch
    height=int(7 / 2.54 * 96),
    margin=dict( # Set the margins
        l=0,  # Left margin
        r=25,  # Right margin
        b=25,  # Bottom margin
        t=25  # Top margin
    ),
    yaxis_title='Coefficient of variance',
    xaxis_title='Log average expression of transcripts'
)

fig.show()

# Save the figure
fig.write_image('../../figures/assembly/variance_vs_log_expression.svg')